### Minimal model for burst noise in transcription-translation

Here we implement a simple model for transcription and translation to study burst noise.

In [ ]:
using Catalyst, DifferentialEquations, Plots, Distributions
using DifferentialEquations.EnsembleAnalysis

First we test if multi-threading is setup correctly. The output of the following cell should be the number of cores in you laptop. For me this is 8.

In [ ]:
Threads.nthreads()

We start by defining the model in the high-level language implemented by the Catalyst.jl module. We consider four reactions:
1. transcription of mRNA
2. degradation of mRNA 
3. translation of mRNA to protein
4. protein degradation

Note 1: the first line in the reaction network definition specifies two reactions, forward and backward, as indictated by the double arrow.

Note 2: in Julia you can use Unicode characters in code, e.g. greek letters, sub- and superscripts. This is very useful to make code more readable and closer to the mathematical representation.

Note 3: The "@" notation signifies a macro, you can read about that in the documentation if you want to know more: https://docs.julialang.org/en/v1/manual/metaprogramming/#man-macros

In [ ]:
coreMDL = @reaction_network begin
    (ρₘ,dₘ), ∅ <--> mRNA          #transcription and degradation of mRNA
    kₚ, mRNA --> mRNA + protein   #translation of mRNA to protein
    dₚ, protein --> ∅             #protein degradation
  end ρₘ dₘ kₚ dₚ
p = [1.0, 0.1, 1.0, 0.01];        #here we define the vector of parameter values ordered as follows: [ρₘ, ρₛ, kₚ, dₚ] - note: you can also use other types to pass the parameters, e.g. Tuple types: p = (1.0, 0.1, 1.0, 0.01)
tspan = (0.,500.);                #here we define the timespan, we don't need to define this here but will need this later
u0 = [0.,0.];                     #here we define the initial condition for our two variables [mRNA, protein]

Next we want to 
- convert the model to a set of ODEs 
- solve the ensuing equations
- plot the results

This gives us the deterministic "mean field" solution to the problem.
I recommend that you have a look at the way plotting works in Julia: https://docs.juliaplots.org/latest/basics/

Note: Ignore the warnings

In [ ]:
# solve ODEs
oprob = ODEProblem(coreMDL, u0, tspan, p);
osol  = solve(oprob, Tsit5());
plot(osol)

Next we want to run a stochastic simulation using the SSA (Gillespie) method. For the documentation & examples see: https://diffeq.sciml.ai/latest/tutorials/discrete_stochastic_example/

Note the use of the exclamation mark in the plot fuction. This plots the data over the previous plot, see the documentation of plot.jl linked above.

In [ ]:
# solve JumpProblem
dprob = DiscreteProblem(coreMDL, u0, tspan, p);
jprob = JumpProblem(coreMDL, dprob, Direct());
jsol = solve(jprob, SSAStepper());
plot!(jsol)  

The previous example only calculated a single solution/trajectory. However, what we want are the statistics, specifically the distribution of protein expression. Hence, we need to run the simulation multiple times. This can be conveniently achieved with Ensemble Simulations, see https://diffeq.sciml.ai/stable/features/ensemble/.

Note: it makes sense to set the number of trajectories to integer multiples of your computers physical CPU cores.

Note: It may be possible to do parameter sweeps with these methods, worth taking a closer look. (I tried a few months ago and couldn't get it working, but could well be I missed something or features were added since.)

In [ ]:
ensemble_prob = EnsembleProblem(jprob)
sim = solve(ensemble_prob,SSAStepper(),EnsembleThreads(),trajectories=18)

In [ ]:
plot(sim, vars=(0,2), inealpha=0.4)

As you can see, all these simulations start from the Null-vector initial condition, i.e. all concentrations are zero at the beginning. This means that we need to run the simuation long enough to reach steady state, since we are interested in the statistics at steady state. However, how long this takes depends on parameters. 

A better approach would be to start the simulation at steady state, see https://diffeq.sciml.ai/stable/types/steady_state_types/

In [ ]:
@named odesys = convert(ODESystem,coreMDL);
prob = ODEProblem(odesys,u0,tspan,p;jac=true,sparse=true)
ssprob = SteadyStateProblem(prob)
ss_sol = round.(solve(ssprob,SSRootfind()))
print(ss_sol)

Now we can use the steady state solution as initial condition for a stochastic simulation, first just a single trajectory.

Note that have now lowered the time span of the simulation.

In [ ]:
# solve JumpProblem
tspan2 = (0.,200.)

dprob2 = DiscreteProblem(coreMDL, ss_sol, tspan2, p);
jprob2 = JumpProblem(coreMDL, dprob2, Direct());
jsol2 = solve(jprob2, SSAStepper());
plot(jsol2)  

And now we can run the ensemble simulation again. 

In [ ]:
function prob_func(prob,i,repeat)
    @. prob.prob.u0 = [rand(Poisson(ss_sol[1])), rand(Poisson(ss_sol[2]))];
    prob
end
ensemble_prob2 = EnsembleProblem(jprob2, prob_func=prob_func);
sim2 = solve(ensemble_prob2,SSAStepper(),EnsembleThreads(),trajectories=16);
plot(sim2, inealpha=0.4)

Even better than starting from a fixed steady state would be to draw the initial condition for each trajectory from a poisson distribution with mean/variance given by the steady state solution computed above (i.ee staring with Fano = 1). It should be possible to do this with the Ensemble Problem - have a look at this.

Next we want to analyse the data, e.g. calculate the Fano factor. We want to make sure that the statistical distribution is stationary, i.e. that the mean and Fano factor do not change anymore with time. Have a think about how you can test that. You will need to calculate both for multiple timepoints. There are some high-level functions that come with the Ensemble analysis, see https://diffeq.sciml.ai/stable/features/ensemble/#Summary-Statistics

However, the timeseries variant does not seem to work, but the since time-point variant does. It is straightforward to implement any analogous fuction. The goal for you is to 
- write a fuction that test if the distribution is stationary (tspan long enough).
- store mean and variance for a particular set of parameters

With that you will need to find a way to do parameter sweeps, i.e. specify a parameter range, e.g. d_m = 0.1, 0.2, ... 1.0, and run simulations to calculate statistics (mean, variance) for each parameter combination. Finally you need to store the results in a convient data format (have a look at DataFrames.jl) and save it to the disk. Then, plot the Fano factor (y) as a function of the parameter (e.g. d_m) and see how it changes.

In [ ]:
timeseries_steps_meanvar(sim2)

In [ ]:
m, v = timepoint_meanvar(sim2,100)